# Setup API Route for Trade Logging
This section demonstrates how to implement an Express.js API route to log trades and retrieve trade history from the trading engine. This enables UI and Colab integration for real-time and batch trading workflows.

In [ ]:
# Express.js API route for trade logging
from flask import Flask, request, jsonify
import json
import os

app = Flask(__name__)
TRADING_LOG = 'trading-log.json'

@app.route('/api/trades', methods=['GET', 'POST', 'DELETE'])
def trades():
    if request.method == 'GET':
        if os.path.exists(TRADING_LOG):
            with open(TRADING_LOG) as f:
                trades = json.load(f)
            return jsonify(trades)
        return jsonify([])
    elif request.method == 'POST':
        trade = request.json
        if os.path.exists(TRADING_LOG):
            with open(TRADING_LOG) as f:
                trades = json.load(f)
        else:
            trades = []
        trades.append(trade)
        with open(TRADING_LOG, 'w') as f:
            json.dump(trades, f, indent=2)
        return jsonify(trade), 201
    elif request.method == 'DELETE':
        with open(TRADING_LOG, 'w') as f:
            json.dump([], f)
        return '', 204

# To run: flask run


# Colab Notebook Example for CSV Import/Export
This section provides Python code for importing and exporting trading datasets in Google Colab. You can use these cells to simulate trades and save results for use in Alpha-Q AI.

In [ ]:
# Import trading dataset and export simulated trades in Colab
import pandas as pd

# Load trading dataset (upload or use sample)
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded)))

# Calculate SMA and simulate trades
window = 3
df['SMA'] = df['Close'].rolling(window=window).mean()
trades = []
for i in range(window, len(df)):
    if df['Close'][i] > df['SMA'][i]:
        trades.append({'timestamp': df['Date'][i], 'type': 'BUY', 'amount': 1, 'price': df['Close'][i], 'result': 'SIMULATED', 'rationale': 'SMA cross'})
    elif df['Close'][i] < df['SMA'][i]:
        trades.append({'timestamp': df['Date'][i], 'type': 'SELL', 'amount': 1, 'price': df['Close'][i], 'result': 'SIMULATED', 'rationale': 'SMA cross'})

# Export simulated trades to CSV
trades_df = pd.DataFrame(trades)
trades_df.to_csv('simulated_trades.csv', index=False)
files.download('simulated_trades.csv')


# UI/UX Enhancements for Trade Visualization
Visualize trade data, price trends, and SMA crossings using Plotly. This helps users and admins understand trading performance and strategy outcomes.

In [ ]:
# Visualize price, SMA, and trades
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Close Price'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA'], mode='lines', name='SMA'))

# Mark trades
for trade in trades:
    color = 'green' if trade['type'] == 'BUY' else 'red'
    fig.add_trace(go.Scatter(
        x=[trade['timestamp']],
        y=[trade['price']],
        mode='markers',
        marker=dict(color=color, size=10),
        name=trade['type']
    ))

fig.update_layout(title='Trading Simulation: Price, SMA, and Trades', xaxis_title='Date', yaxis_title='Price')
fig.show()
